# Mean encodings

В этом домашнем задании вы будете работать с выборкой `1C`. Вам нужно посчитать счетчики для `item_id` четырьмя способами:

    1) При помощи KFold схемы;  
    2) При помощи Leave-one-out схемы;
    3) При помощи smoothing схемы;
    4) При помощи expanding mean схемы.

### Подсказки

* NAN-ы следует заменить на `0.3343`.
* Некоторые схемы mean encoding зависят от порядка сортировки. Для того, чтобы получить правильный ответ, используейте следующий фрагмент кода, чтобы построить выборку. Этот код также делает mean encoding без какой-либо регуляризации.

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from sklearn.model_selection import KFold

# Считываем данные

In [2]:
sales = pd.read_csv('sales_train.csv.gz')
sales.columns = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'target']
sales

,date,date_block_num,shop_id,item_id,item_price,target
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


# Агрегируем данные

Нас интересуют месячные предсказания, поэтому перед кодированием необходимо агрегировать данные на месячном уровне. Следующий фрагмент кода это делает. 

In [3]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'target':'sum'})

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

# Mean encodings без регуляризации

После проделанной технической работы, мы готовы посчитать счетчики для переменной `item_id`. 

Ниже приведены две реализации подсчета счетчиков без регуляризации. Можно использовать данный код в качестве стартовой точки для реализации различных техник регуляризации.

#### Способ 1

In [4]:
all_data.groupby('item_id')['target'].mean()

item_id
0        0.020000
1        0.023810
2        0.019802
3        0.019802
4        0.020000
           ...   
22165    0.021277
22166    0.295918
22167    1.081081
22168    0.032967
22169    0.020833
Name: target, Length: 21807, dtype: float64

In [5]:
# Calculate a mapping: {item_id: target_mean}

item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(encoded_feature)
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])
print(item_id_target_mean)

[0.02222222 0.05683356 0.14117647 ... 1.23558897 0.29591837 1.08108108]
0.4830386988621699
item_id
0        0.020000
1        0.023810
2        0.019802
3        0.019802
4        0.020000
           ...   
22165    0.021277
22166    0.295918
22167    1.081081
22168    0.032967
22169    0.020833
Name: target, Length: 21807, dtype: float64


#### Способ 2

In [6]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621699


# 1. KFold схема

Необходимо реализовать Kfold схему с пятью фолдами (**3 балла**). Используйте KFold(5) из sklearn.model_selection. 

1. Разбейте данные на 5 фолдов при помощи `sklearn.model_selection.KFold` с параметром `shuffle=False`.
2. Проитерируйтесь по фолдам: используйте 4 обучающих фолда для подсчета средних значений таргета по `item_id` и заполните этими значениями валидационный фолд на каждой итерации.

Обратите внимание на **Способ 1** из примера. В частности, изучите, как работают функции map и pd.Series.map. Они довольно полезны во многих ситуациях. 

In [7]:
kf = KFold(n_splits=5, shuffle = False)
X = all_data.copy()
for train_index, valid_index in kf.split(X):
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        item_id_target_mean = X_train.groupby('item_id')['target'].mean()
        X.loc[X.index[valid_index], 'item_target_enc'] = X_valid['item_id'].map(item_id_target_mean)
        X['item_target_enc'].fillna(0.3343, inplace=True) 

# You will need to compute correlation like that
encoded_feature = X['item_target_enc'].values
corr = np.corrcoef(X['target'].values, encoded_feature)[0][1]
print(round(corr, 4))

0.4165


Ожидаемый ответ 0.4165

# 2. Leave-one-out схема

Необходимо реализовать leave-one-out схему (**3 балла**). Учтите, если вы запустите код из первого задания, задав количество фолдов такое же как размер выборки, то вы, вероятно, получите правильный ответ, но ждать будете очень-очень долго.

Для более быстрой реализации подсчета среднего таргета на всех объектах, кроме одного, вы можете:

1. Вычислить суммарный таргет по всем объектам.
2. Вычесть таргет конкретного объекта и разделить результат на `n_objects - 1`. 

Заметим, что пункт `1.` следует сделать для всех объектов. Также заметим, что пункт `2.` может быть реализован без циклов `for`.

Здесь может оказаться полезной функция .transform из **Способа 2** из примера.

In [9]:
LOO_data = all_data.copy()
target = LOO_data['target']
series_sum = LOO_data.groupby('item_id')['target'].transform('sum')
n_objects = LOO_data.groupby('item_id')['target'].transform('count')
series = (series_sum - target) / (n_objects - 1)
LOO_data['item_target_enc'] = series
encoded_feature = LOO_data['item_target_enc'].values
# Fill NaNs
LOO_data['item_target_enc'].fillna(0.3343, inplace=True) 
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(round(corr, 5))

0.48038


Ожидаемый ответ 0.4803

# 3. Smoothing

Необходимо реализовать smoothing с $\alpha = 100$ (**3 балла**). Используйте формулу:

$\frac{mean(target) \cdot nrows + globalmean \cdot \alpha }{nrows + \alpha}$,

где $globalmean=0.3343$. Заметим, что `nrows` - это количество объектов, принадлежащих конктертной категории, а не количество строк в датасете.

In [18]:
smooth_data = all_data.copy()
alpha = 100
globalmean = 0.3343
nrows = smooth_data.groupby('item_id')['target'].transform("count")
mean_target = smooth_data.groupby('item_id')['target'].transform("mean")

smooth = (mean_target * nrows + globalmean * alpha) / (nrows + alpha)
smooth_data['item_target_enc'] = (mean_target * nrows + globalmean * alpha) / (nrows + alpha)

smooth_data['item_target_enc'].fillna(0.3343, inplace=True) 
encoded_feature = smooth_data['item_target_enc'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(round(corr, 4))

0.4818


Ожидаемый ответ 0.4818

# 4. Expanding mean схема

Необходимо реализовать *expanding mean* схему (**4 балла**). Ее суть заключается в том, чтобы пройти по отсортированному в определенном порядке датасету (датасет сортируется в самом начале задания) и для подсчета счетчика для строки $m$ использовать строки от $0$ до $m-1$. Вам будет необходимо воспользоваться pandas функциями [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) и [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html).

In [27]:
EM = all_data.copy()
target = EM['target']
EM['item_target_enc'] = (X.groupby('item_id')['target'].cumsum(axis = 0) - target) /  EM.groupby('item_id')['target'].cumcount()
EM['item_target_enc'].fillna(0.3343, inplace=True) 

encoded_feature = EM['item_target_enc'].values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(round(corr, 4))

0.5025


Ожидаемый ответ 0.5025